# Air Sac Tracking Prototype code
Lara S. Burchardt & Wim Pouw

lara.burchardt@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [1]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv
import random2
import statistics
import scipy
from scipy import signal
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import itertools
import os
from os import listdir
from os.path import isfile, join


#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

## Define Prepocessing Function

In [2]:
#define parameters for HoughTransform outside of function to be able to save and manipulate easier
def hougdraw(submitted_image, dp = 1, mindist = 10000, param1 = 10, param2=22, minradius = 5, maxradius=250):
    circles = cv2.HoughCircles(submitted_image, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = mindist,  
                               param1 = param1, param2 = param2, 
                               minRadius = minradius, maxRadius = maxradius)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        circle1 = circles[0,0]
        circle2 = circles[0,1]
        circle3 = circles[0,2]
        for(x, y, r) in circles:
            cv2.circle(submitted_image, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image 
    return(submitted_image)
    
# define function for preprocessing
def preprocessing(image, phase1_medianblur = 27, 
                              phase2_dilate = 7, 
                              phase2_medianblur = 19, 
                              alpha = 2, 
                              beta= 30,
                              param1canny=10,
                              param2canny=15):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/param1canny))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/param2canny))    
    image1_h = hougdraw(gray,param1= threshold1, param2 =threshold2)
    #blur
    image2 = cv2.medianBlur(gray, phase1_medianblur)
    image2_h = hougdraw(image2.copy(),param1= threshold1, param2 =threshold2)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    image3_h = hougdraw(image3.copy(), param1= threshold1, param2 =threshold2)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= phase2_dilate)  
    image4 = cv2.medianBlur(submitted, phase2_medianblur) 
    image4_h = hougdraw(image4.copy(), param1= threshold1, param2 =threshold2)
    #add hough
    return image1_h, image2_h, image3_h, image4_h

def matriximages(outputname, wimagenumber, himagenumber, imagelist, marginbetweenimages, vertnames, hornames):
    """
    matrix images takes in the full path name of the outputfolder for storing the images
    it takes as input a list of full path names of the images
    the height is given in image numbers and same for width
    the vert names is a list of column names
    the hor names is a list of row names
    """
    margin = marginbetweenimages
    #img = imagelist  
    w = wimagenumber
    h = himagenumber
    n = w*h

    #imgs = [cv2.imread(i) for i in imagelist]
    imgs = imagelist
    #if any(i.shape != imgs[0].shape for i in imgs[1:]):
    #    raise ValueError('Not all images have the same shape.')

    img_h, img_w = imgs[0].shape#img_h, img_w, img_c = imgs[0].shape

    m_x = 0
    m_y = 0
    if marginbetweenimages is not None:
        margin = marginbetweenimages
        m_x = int(margin*img_w)
        m_y = int(margin*img_h)
        
    imgmatrix = np.zeros((img_h * h + m_y * (h - 1),
                          img_w * w + m_x * (w - 1)),
                         np.uint8)

    imgmatrix.fill(255)    

    positions = itertools.product(range(w), range(h))
    for (x_i, y_i), img in zip(positions, imgs):
        x = x_i * (img_w + m_x)
        y = y_i * (img_h + m_y)
        imgmatrix[y:y+img_h, x:x+img_w] = img #imgmatrix[y:y+img_h, x:x+img_w, :] = img
    #add text
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(imgmatrix, hornames[0], (0*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[1], (1*img_w,  int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[2], (2*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[3], (3*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[4], (4*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[0], (0, 1*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[1], (0, 2*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[2], (0, 3*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[3], (0, 4*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    #cv2.putText(imgmatrix, vertnames[4], (0, 5*img_h), font, 10, (0, 255, 0), 5, cv2.LINE_AA)
    cv2.imwrite(outputname, imgmatrix)   
    
    print('done, look in your folder: '+ outputname)

In [38]:


#####################loading in the images
imagefolder = '../TestFrames/testframes_9112022/'
ims = [f for f in listdir(imagefolder) if isfile(join(imagefolder, f))]
imlist = []
for i in ims: #add the image folder name to get the full path
    imlist.append(imagefolder +i) 
    
imgs = [cv2.imread(i) for i in imlist] #laod in the images

####################
imagesprocessed = []
for imtoprocess in imgs:
    print("processing image")
    image1, image2, image3, image4 = preprocessing(image= imtoprocess)
    imagesprocessed.append(image1)
    imagesprocessed.append(image2)
    imagesprocessed.append(image3)
    imagesprocessed.append(image4)
    print("one image completed! - > so happyyyyyy")
print('done')

processing image
one image completed! - > so happyyyyyy
processing image
one image completed! - > so happyyyyyy
processing image
one image completed! - > so happyyyyyy
processing image
one image completed! - > so happyyyyyy
processing image
one image completed! - > so happyyyyyy
done


In [7]:
#THIS IS JUST TO TEST THE DYNAMIC THRESHOLD

#import os
#from os import listdir
#from os.path import isfile, join
#
######################loading in the images
#imagefolder = '../TestFrames/testframes_9112022/'
#ims = [f for f in listdir(imagefolder) if isfile(join(imagefolder, f))]
#imlist = []
#for i in ims: #add the image folder name to get the full path
#    imlist.append(imagefolder +i) 
#    
#imgs = [cv2.imread(i) for i in imlist]
#
#gray = cv2.cvtColor(imgs[0], cv2.COLOR_RGB2GRAY)
#    #brightness change
#gray = cv2.convertScaleAbs(gray, alpha = 2, beta = 30)  
#    #blur
#image2 = cv2.medianBlur(gray, 27)
#
##dynamic thresholds for canny edge detection based on intensity of image
#mean_intensity = np.median(gray)
#check dynamic tresholds
#print(str(int(max(0, (1.0 - 0.33) * mean_intensity/14))) + 'should be 10')#threshold1 = #10#int(max(0, (1.0 - 0.33) * mean_intensity))
#print(str(int(min(255, (1.0 + 0.33) * mean_intensity/14))) + 'should be 15')
#threshold2 = #15#int(min(255, (1.0 + 0.33) * mean_intensity))



In [39]:
#now bind them into one figure with some column names
outputf = os.path.abspath('./Output/test.jpeg')
cnames = ['example1', 'example2', 'example3', 'example4', 'example5']
rnames = ['original', 'blur', 'canny','dilation']

matriximages(outputname = outputf, wimagenumber =5, himagenumber = 4, imagelist=imagesprocessed, marginbetweenimages=0,vertnames =rnames, hornames= cnames)


done, look in your folder: D:\Research_Projects\workspace_airsac20\AirSacTracker\Script\Output\test.jpeg


# Loop through thresholds and generate preprocessing matriximages

In [ ]:
#####################loading in the images
imagefolder = '../TestFrames/testframes_9112022/'
ims = [f for f in listdir(imagefolder) if isfile(join(imagefolder, f))]
imlist = []
for i in ims: #add the image folder name to get the full path
    imlist.append(imagefolder +i) 
imgs = [cv2.imread(i) for i in imlist] #laod in the images
#######################################

#
alphas = [1, 1.5, 2]
betas = [10, 20, 30]
dilations = [5,6,7]
phase1_medianblurs = [13, 19, 27]
cannyt1s = [5, 7, 10, 12, 14] #which amounts to static t1 [21, 15 , 10, 7]
cannyt2s = [5, 7, 10, 12, 14] #which amounts to static t2 [42, 30, 21, 15]

#loop through all
for cannyt1 in cannyt1s:
    for cannyt2 in cannyt2s:
        for alpha in alphas:
            for beta in betas:
                for dilation in dilations:
                    for phase1_medianblur in phase1_medianblurs:
                        if cannyt1 < cannyt2: #canny 2 needs to be lower
                            outputcode = 'c1_'+str(cannyt1)+'_c2_'+\
                                str(cannyt2)+'_al_'+str(alpha)+'_b_'+str(beta)+'_dil_'+\
                                str(dilation)+'_blur_'+str(phase1_medianblur)
                            print('working on ' + outputcode)
                            outputf = os.path.abspath('./Output/'+outputcode+'.jpeg')
                            if os.path.exists(outputf) == False:
                                cnames = ['example1', 'example2', 'example3', 'example4', 'example5']
                                rnames = ['original_'+str(alpha)+'_'+str(beta), 'blur_' + str(phase1_medianblur), 'canny_' + str(cannyt1) + '_' + str(cannyt2),'dilation_' + str(dilation)]
                                ####################DO ROUTINE
                                imagesprocessed = []
                                for imtoprocess in imgs:
                                    image1, image2, image3, image4 = preprocessing(image= imtoprocess,
                                        phase1_medianblur = phase1_medianblur, 
                                          phase2_dilate = dilation, 
                                          phase2_medianblur = 17, #keep unchanges
                                          alpha = alpha, 
                                          beta= beta,
                                          param1canny=cannyt1,
                                          param2canny=cannyt2)
                                    imagesprocessed.append(image1)
                                    imagesprocessed.append(image2)
                                    imagesprocessed.append(image3)
                                    imagesprocessed.append(image4)
                                    print("processed image")
                                #plot images
                                matriximages(outputname = outputf, wimagenumber =5, himagenumber = 4, imagelist=imagesprocessed, marginbetweenimages=0,vertnames =rnames, hornames= cnames)


working on c1_5_c2_7_al_1_b_10_dil_5_blur_13
working on c1_5_c2_7_al_1_b_10_dil_5_blur_19
working on c1_5_c2_7_al_1_b_10_dil_5_blur_27
working on c1_5_c2_7_al_1_b_10_dil_6_blur_13
working on c1_5_c2_7_al_1_b_10_dil_6_blur_19
working on c1_5_c2_7_al_1_b_10_dil_6_blur_27
working on c1_5_c2_7_al_1_b_10_dil_7_blur_13
working on c1_5_c2_7_al_1_b_10_dil_7_blur_19
working on c1_5_c2_7_al_1_b_10_dil_7_blur_27
working on c1_5_c2_7_al_1_b_20_dil_5_blur_13
working on c1_5_c2_7_al_1_b_20_dil_5_blur_19
working on c1_5_c2_7_al_1_b_20_dil_5_blur_27
working on c1_5_c2_7_al_1_b_20_dil_6_blur_13
working on c1_5_c2_7_al_1_b_20_dil_6_blur_19
working on c1_5_c2_7_al_1_b_20_dil_6_blur_27
working on c1_5_c2_7_al_1_b_20_dil_7_blur_13
working on c1_5_c2_7_al_1_b_20_dil_7_blur_19
working on c1_5_c2_7_al_1_b_20_dil_7_blur_27
working on c1_5_c2_7_al_1_b_30_dil_5_blur_13
working on c1_5_c2_7_al_1_b_30_dil_5_blur_19
working on c1_5_c2_7_al_1_b_30_dil_5_blur_27
working on c1_5_c2_7_al_1_b_30_dil_6_blur_13
working on

processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_1.5_b_20_dil_5_blur_27.jpeg
working on c1_5_c2_10_al_1.5_b_20_dil_6_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_1.5_b_20_dil_6_blur_13.jpeg
working on c1_5_c2_10_al_1.5_b_20_dil_6_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_1.5_b_20_dil_6_blur_19.jpeg
working on c1_5_c2_10_al_1.5_b_20_dil_6_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_1.5_b_20_dil_6_blur_27.jpeg
working on c1_5_c2_10_al_1.5_b_20_dil_7_blur

processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_2_b_20_dil_7_blur_27.jpeg
working on c1_5_c2_10_al_2_b_30_dil_5_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_2_b_30_dil_5_blur_13.jpeg
working on c1_5_c2_10_al_2_b_30_dil_5_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_2_b_30_dil_5_blur_19.jpeg
working on c1_5_c2_10_al_2_b_30_dil_5_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_10_al_2_b_30_dil_5_blur_27.jpeg
working on c1_5_c2_10_al_2_b_30_dil_6_blur_13
processed im

processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_1_b_30_dil_6_blur_27.jpeg
working on c1_5_c2_12_al_1_b_30_dil_7_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_1_b_30_dil_7_blur_13.jpeg
working on c1_5_c2_12_al_1_b_30_dil_7_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_1_b_30_dil_7_blur_19.jpeg
working on c1_5_c2_12_al_1_b_30_dil_7_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_1_b_30_dil_7_blur_27.jpeg
working on c1_5_c2_12_al_1.5_b_10_dil_5_blur_13
processed image
processed image
processed image
processed 

processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_2_b_10_dil_5_blur_27.jpeg
working on c1_5_c2_12_al_2_b_10_dil_6_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_2_b_10_dil_6_blur_13.jpeg
working on c1_5_c2_12_al_2_b_10_dil_6_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_2_b_10_dil_6_blur_19.jpeg
working on c1_5_c2_12_al_2_b_10_dil_6_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_12_al_2_b_10_dil_6_blur_27.jpeg
working on c1_5_c2_12_al_2_b_10_dil_7_blur_13
processed im

processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1_b_10_dil_7_blur_27.jpeg
working on c1_5_c2_14_al_1_b_20_dil_5_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1_b_20_dil_5_blur_13.jpeg
working on c1_5_c2_14_al_1_b_20_dil_5_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1_b_20_dil_5_blur_19.jpeg
working on c1_5_c2_14_al_1_b_20_dil_5_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1_b_20_dil_5_blur_27.jpeg
working on c1_5_c2_14_al_1_b_20_dil_6_blur_13
processed image
processed image
processed image
processed im

processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1.5_b_20_dil_6_blur_27.jpeg
working on c1_5_c2_14_al_1.5_b_20_dil_7_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1.5_b_20_dil_7_blur_13.jpeg
working on c1_5_c2_14_al_1.5_b_20_dil_7_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1.5_b_20_dil_7_blur_19.jpeg
working on c1_5_c2_14_al_1.5_b_20_dil_7_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_1.5_b_20_dil_7_blur_27.jpeg
working on c1_5_c2_14_al_1.5_b_30_dil_5_blur_13
processed image
processed im

processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_2_b_30_dil_5_blur_27.jpeg
working on c1_5_c2_14_al_2_b_30_dil_6_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_2_b_30_dil_6_blur_13.jpeg
working on c1_5_c2_14_al_2_b_30_dil_6_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_2_b_30_dil_6_blur_19.jpeg
working on c1_5_c2_14_al_2_b_30_dil_6_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_5_c2_14_al_2_b_30_dil_6_blur_27.jpeg
working on c1_5_c2_14_al_2_b_30_dil_7_blur_13
processed image
processed image
processed im

done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_7_c2_10_al_1_b_30_dil_7_blur_27.jpeg
working on c1_7_c2_10_al_1.5_b_10_dil_5_blur_13
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_7_c2_10_al_1.5_b_10_dil_5_blur_13.jpeg
working on c1_7_c2_10_al_1.5_b_10_dil_5_blur_19
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_7_c2_10_al_1.5_b_10_dil_5_blur_19.jpeg
working on c1_7_c2_10_al_1.5_b_10_dil_5_blur_27
processed image
processed image
processed image
processed image
processed image
done, look in your folder: E:\research_airsactracker\AirSacTracker\Script\Output\c1_7_c2_10_al_1.5_b_10_dil_5_blur_27.jpeg
working on c1_7_c2_10_al_1.5_b_10_dil_6_blur_13
processed image
processed image
processed image
processed image
processed imag

## Define ROI automatically

The tracker is run on a random subset of 50 frames of the video, the median position and size of circles is used to
define a Region of Interest (ROI) for the full analysis


In [11]:
# preprocessing, choosing a roi on the tracking results of 50 random samples from input file

#https://github.com/patchy631/machine-learning/blob/main/computer_vision/cv2_edge_detection.ipynb

videofolder= '../Video/'
videofilename = 'June12_02.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second


#set up empty output dataframe
column_names = ['x','y', 'r', 'frame'] # what do we need to save from this?  
df_round_1 = pd.DataFrame(columns = column_names)

# initialize parameters for pre-processing (used in cv2.convertScaleAbs)
alpha = 2 #1.5
beta = 20 #30

#croppedFrame = frame[int(roi[1]): int(roi[1]+roi[3]),
#                     int(roi[0]):int(roi[0]+roi[2])] 

#we define a sequence of 50 ints first, with random numbers of "totalFrames" using sample() because that is sampling without replacement
# then we loop over that list of numbers
# we need an if to check, whether there is >= 50 frames in the video

vector = range(0, int(totalFrames), 1)
samp = random2.sample(vector, 50)
#samp = random2.sample(vector, int(totalFrames))

for rand in samp:
  
    # set frame position

    cap.set(cv2.CAP_PROP_POS_FRAMES,rand)
    ret, frame = cap.read()
  
    ############################detect circles   
    output=frame.copy()
    # transform to grayscale image only using the roi part of the image
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    output_gray=gray.copy()
    # increasing brightness and contrast
    gray_light = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    # Calculate mean intensity
    mean_intensity = np.median(gray)
    # Set thresholds to be one standard deviation above and below median intensity
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity))
    
    gray = cv2.medianBlur(gray_light, 19)
  
    # Apply canny edge detector with dynamic threshold
    submitted = cv2.Canny(gray, threshold1, threshold2)
    submitted = cv2.dilate(submitted, None, iterations=9)  
    submitted = cv2.GaussianBlur(submitted,(7,7),0) 
    
    #track demicircles 
    # define parameters for HoughTransform outside of function to be able to save and manipulate easier
    dp = 1
    minDist = 10000
    param1 = 10
    param2 = 22 # war 10
    minRadius = 5
    maxRadius = 250

    circles = cv2.HoughCircles(submitted, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, 
                               minRadius = minRadius, maxRadius = maxRadius)
    #if circles is not None:
    #    circles = np.round(circles[0, 0:1]).astype("int")
    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles[0, 0:1]).astype("int")
            circle1 = circles[0,0]
            circle2 = circles[0,1]
            circle3 = circles[0,2]
        #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
    cv2.waitKey(1)

    new_row = [circles[0,0], circles[0,1],circles [0,2], rand]
    
    df_round_1.loc[len(df_round_1)] = new_row
    
    median_x = df_round_1['x'].median() #order in pos : same as in original dataframe, so x,y, r
    median_y = df_round_1['y'].median()
    max_r = df_round_1['r'].max()
    min_r = df_round_1['r'].min()
    # order of parameters saved as roi with cv2.selectROI [Top_Left_X, Top_Left_Y, Width, Height]
   
    # explanation: height_1 is the maximum radius detected plus 30 pixels, that is used to determine the position of the 
    # upper left corner of the roi, same goes for width_1, both are currently the same, but that could change, so they are both
    # coded independently
    height_1 = max_r + 30
    width_1 = max_r + 30
    pos_x = median_x - width_1
    pos_y = median_y - height_1
    
    # to be consistent with the roi nomenclature, we then calculate the width/height of the whole roi which is width_1 *2
    # all those values will then be used to crop picture in next round of tracking
    
    width_2 = width_1 * 2
    height_2 = height_1 *2
    
    
            
# cleaning up
#out.release()
cap.release()
cv2.destroyAllWindows()

## Main Function

The tracker is run on the full video, within the ROI defined in the first step. 
The output is saved in a table format and as a video with the detected circles drawn on top. 

In [13]:
videofolder= '../Video/'
videofilename = 'June12_02.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#output video
out = cv2.VideoWriter('./Output/output_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))
out_2 = cv2.VideoWriter('./Output/output_processed_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))
#set up empty output dataframe
column_names = ['frame','roi_x1','roi_y1','roi_x2', 'roi_y2', # info on region of interest for repetability
                'x','y', 'r', 'inputvideo','input_id',                    # actual results 
                'alpha', 'beta',                              # values of brightness and contrast pre processing
                'dp', 'minDist', 'param1', 'param2', 'minRadius',' maxRadius'] # parameters of hough circle transform 
df = pd.DataFrame(columns = column_names)

#initialize iterator
i=0

#ROI as automatically detected beforehand
roi = [pos_x, pos_y, width_2, height_2]

#main loop                      
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
          
  ############################detect circles   
    
    output=frame.copy()
    
    image1,image2,image3,image4 = preprocessing(image = frame, phase1_medianblur = 19, phase2_dilate = 7, phase2_medianblur = 19)
    
    #track demicircles 
    # define parameters for HoughTransform outside of function to be able to save and manipulate easier
    
    dp = 1
    minDist = 10000
    param1 = 10
    param2 = 22 # was 10
    minRadius = min_r - 10 #minus 2times std
    maxRadius = 270 #dynamic? 
    name = 'framenr_' + str(i+1) + '_framevid_' + videofilename[0:-4] 
    
    # original values: dp = 1, minDist = 10000, param1=1, param2=10, minRadius = 5, maxRadius= 250
    circles = cv2.HoughCircles(image4, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, 
                               minRadius = minRadius, maxRadius = maxRadius)
    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles[0, 0:1]).astype("int")
            circle1 = circles[0,0]
            circle2 = circles[0,1]
            circle3 = circles[0,2]
        #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
            
        # code for drawing the found circles onto the original video    
        for (x, y, r) in circles:
            cv2.circle(output, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image
            #cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
            #         int(roi[0]):int(roi[0]+roi[2])], (x, y), r, (255, 255, 0), 2)
            cv2.circle(image4, (x, y), r, (200, 0, 0), 2)
            #print(x,y,r)
    cv2.waitKey(1)
    out.write(output)
    #out_2.write(image5)
    cv2.imshow("output", submitted)
    i=i+1

# saving results and used parameters row wise during loop into a dataframe

    new_row = [i+1, roi[0],roi[1], roi[2], roi[3], circle1, circle2,circle3, videofilename, name,
               alpha, beta, dp, minDist, param1, param2, minRadius, maxRadius]
    
    df.loc[len(df)] = new_row

# filename and saving dataframe as cvs file       
filename =  'airsacradius_results_' + videofilename + '.csv'
df.to_csv(filename, sep = ',')

# cleaning up
out.release()
cap.release()
cv2.destroyAllWindows()

# Smoothing

Detected circle position and radius get smoothed, using Savitzky-Golay filter

In [22]:
#smoothing of x,y and r of circles
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html
#read raw data
videofolder= '../Video/'
videofilename = 'June12_02.mp4'
filename =  'airsacradius_results_' + videofilename + '.csv'
dfraw = pd.read_csv(filename)
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
fps = cap.get(cv2.CAP_PROP_FPS) 
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

#outputfile smoothed
out_smoothed = cv2.VideoWriter('./Output/output_smoothed'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))


#delete outliers
#how? dynamic? mean +- 2*std of position of centroid in x or y direction? everything under or over that will be deleted
# calculate euclidean distance? to take x and y into account? 

# filter version using butterworth filter
#b, a = scipy.signal.butter(8, 3,'low', fs = 25)
        
#smooth_x = scipy.signal.filtfilt(b, a, dfraw['x'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)  
#smooth_y = scipy.signal.filtfilt(b, a, dfraw['y'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)
#smooth_r = scipy.signal.filtfilt(b, a, dfraw['r'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)

#filter version using Savitzky-golay filter
wl = 11 #windowlength for savgol filter, must be odd
p = 2   # polynomial for savgol filter, must be less than window_length

smooth_x = savgol_filter(dfraw['x'], window_length=wl, polyorder=p)
smooth_y = savgol_filter(dfraw['y'], window_length=wl, polyorder=p)
smooth_r = savgol_filter(dfraw['r'], window_length=wl, polyorder=p)


circles_draw = list(np.column_stack((smooth_x, smooth_y, smooth_r)))

name = dfraw['input_id']
circles_save = list(np.column_stack((smooth_x, smooth_y, smooth_r, name)))
raw_circles = list(np.column_stack((dfraw['x'], dfraw['y'], dfraw['r'])))

column_names = ['x', 'y', 'r', 'inputfile']


df2 = pd.DataFrame(circles_save, columns = column_names)
filename =  'airsacradius_results_smoothed' + videofilename + '.csv'
df2.to_csv(filename, sep = ',')

i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret == False:
        break
    output=frame.copy()
    x,y,r = circles_draw[i]
    a,b,c = raw_circles[i]
    #cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
    #                 int(roi[0]):int(roi[0]+roi[2])], (int(x), int(y)), int(r), (255, 255, 0), 2)
    cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
                     int(roi[0]):int(roi[0]+roi[2])],(int(x), int(y)), int(r), (255, 255, 0), 1) #blue circle smoothed (color code is the three integers in round brackets)
    cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
                     int(roi[0]):int(roi[0]+roi[2])],(int(a), int(b)), int(c), (0, 0, 255), 1)   # red circle raw
    #coordinates to draw roi, NOT SURE ROI IS WORKING RIGHT NOW!
    cv2.rectangle(output,(int(roi[0]), int(roi[1])),
                     (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 2)
    i= i+1
            
    #cv2.imshow("output", output)
    out_smoothed.write(output)

    
    
# cleaning up
out_smoothed.release()
cap.release()
cv2.destroyAllWindows()

#other filter option: savgoy filter
#from scipy.signal import savgol_filter
#def savgol(x, wl=14, p=2):
#    return savgol_filter(x, window_length=wl, polyorder=p)


# to do: filtering is too heavy with (8,0.125) check other combinations, also check whether filter is set correctly for
# our sampling rate --> changed that, fs is now in the function, what does the 8 mean? 
# automatic roi seems to be amiss quite heavily for all videos but 16_20...check again how to improve
# we need to read in roi from the data sheet, otherwise weird things can happen

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\drawing.cpp:1886: error: (-215:Assertion failed) radius >= 0 && thickness <= MAX_THICKNESS && 0 <= shift && shift <= XY_SHIFT in function 'cv::circle'
